In [97]:
import os
import datetime
import pandas as pd
import sqlalchemy
import pythoncom
import win32com.client as win32
import numpy as np 
from dotenv import load_dotenv
load_dotenv(override=True)
import locale

try:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
except locale.Error:
    locale.setlocale(locale.LC_TIME, 'Spanish')

In [98]:
MYSQL_PASSWORD = os.getenv("DB_PASSWORD")
MYSQL_HOST = os.getenv("DB_HOST") 
MYSQL_PORT = "3306"
MYSQL_DATABASE = os.getenv("DB_NAME")
MYSQL_DRIVER = "mysql+mysqlconnector"
MYSQL_USER= os.getenv("DB_USER")

engine = sqlalchemy.create_engine(
    f"{MYSQL_DRIVER}://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}"
)

with open(r"C:\Users\gpavez\Desktop\Compensaciones\git\compensaciones\querys\query_cumpleaños.sql", "r", encoding="utf-8") as f:
    query_cumpleaños = f.read()

df = pd.read_sql(query_cumpleaños, engine)

In [99]:
df.columns

Index(['nombre', 'rut', 'cumpleaños', 'área', 'dirección', 'empresa',
       'marcaje'],
      dtype='object')

In [1]:
df['cumpleaños'] = pd.to_datetime(df['cumpleaños'], errors='coerce')
df.dropna(subset=['cumpleaños'], inplace=True)
today = datetime.date.today()

# if today.month == 12:
#     next_month = 1
#     next_month_year = today.year + 1
# else:
#     next_month = today.month + 1
#     next_month_year = today.year

current_month = today.month
current_month_year = today.year

#birthdays_next_month_df = df[df['cumpleaños'].dt.month == next_month].copy()
birthdays_next_month_df = df[df['cumpleaños'].dt.month == current_month ].copy() #cumpleaños este mes

birthdays_next_month_df['day'] = birthdays_next_month_df['cumpleaños'].dt.day
birthdays_next_month_df = birthdays_next_month_df.sort_values(by='day')

birthdays_next_month_df.columns

condicion = birthdays_next_month_df['marcaje'].str.contains('las encinas', case=False, na=False)

birthdays_next_month_df['marca'] = np.where(condicion, 'Las Encinas', 'Lucerna')

print(birthdays_next_month_df[['marcaje', 'marca']].head())


NameError: name 'pd' is not defined

In [101]:
def create_html_body(df_birthdays, month_name):
    if df_birthdays.empty:
        return None
    
    html_style = """
    <style>
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #333; }
        h1 { color: #0056b3; }
        h2 { color: #004085; border-bottom: 2px solid #ccc; padding-bottom: 5px; }
        ul { list-style-type: none; padding-left: 0; }
        li { background-color: #f8f9fa; margin: 5px 0; padding: 10px; border-left: 5px solid #0056b3; }
        .footer { font-size: 0.8em; color: #777; margin-top: 20px; }
    </style>
    """
    html_content = f"<html><head>{html_style}</head><body>"
    html_content += f"<h1>🎂 Cumpleaños del mes de {month_name.capitalize()}</h1>"
    html_content += "<p>A continuación, el listado de colaboradores que estarán de cumpleaños el próximo mes:</p>"

    for group_name, group_df in df_birthdays.groupby('marca'):
        html_content += f"<h2>📍 {group_name}</h2>"
        html_content += "<ul>"
        for index, row in group_df.iterrows():
            birthday_str = row['cumpleaños'].strftime('%d / %B')
            html_content += f"<li><b>{row['nombre']}</b> - {row['área']} - {birthday_str}</li>"
        html_content += "</ul>"
    
    html_content += "<p class='footer'>Este es un correo generado automáticamente.</p>"
    html_content += "</body></html>"
    return html_content

# Obtener el nombre del mes en español
#next_month_date = datetime.date(next_month_year, next_month, 1)
next_month_date = datetime.date(current_month_year, current_month, 1)

next_month_name = next_month_date.strftime('%B')

# Generar el cuerpo del correo UNA SOLA VEZ
html_body = create_html_body(birthdays_next_month_df, next_month_name)

In [102]:
def send_email_outlook(subject, html_body_content):
    """Envía un correo usando la aplicación de Outlook local."""
    
    mail_prueba = "gpavez@cramer.cl"
    mail_nicole = "navalos@cramer.cl"
    try:
        pythoncom.CoInitialize()
        
        outlook = win32.Dispatch("outlook.application")
        mail = outlook.CreateItem(0)     
        mail.To = mail_nicole
        mail.Subject = subject
        mail.HTMLBody = html_body_content
        mail.Send()

        pythoncom.CoUninitialize()
    
        print(f"✅ Email enviado exitosamente a: {mail_prueba}")
        
    except Exception as e:
        print(f"❌ Error enviando correo con Outlook: {e}")

In [103]:
if html_body:
    email_subject = f"🎉 Cumpleaños de {next_month_name.capitalize()}"
    
    send_email_outlook(email_subject, html_body)
else:
    print(f"No hay cumpleaños para el mes de {next_month_name}, no se enviará correo.")


✅ Email enviado exitosamente a: gpavez@cramer.cl
